In [ ]:
import os
import sys
from io import open
import unicodedata
import random
import time
import string
import math
import glob

#torch
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

#plot
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#use "$export CUDA_VISIBLE_DEVICES=2,3,..." to assign GPU
#device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device=torch.device("cpu")

In [ ]:
#small Functions

def toTensor(tokLst):#tokLst: a numpy or python list of numbers
    return torch.tensor(tokLst, dtype=torch.long, device=device)

def randomTrainingData(tNames):
    tName= tNames[random.randint(0,len(tNames)-1)]
    inputT = toTensor(tName[:-1])
    targetT = toTensor(tName[1:])
    return inputT,targetT

def stringLstReverse(stringLst):
    return [s[::-1] for s in stringLst]

def reverseNameGenerate(fileName):
    #baseName= (os.path.basename(fileName))
    #dirName = os.path.dirname(fileName)
    #revFileName = dirName+'/'+baseName+'.reverse'
    revFileName=fileName+'.reverse'
    print(revFileName)
    revFile = open(revFileName,'w')
    file = open(fileName,'r')
    for line in file:
        revFile.write(line[:-1][::-1]+'\n')
    file.close()
    revFile.close()
    return revFileName
    

# Model

In [ ]:
class RNN(nn.Module):
    def __init__(self,inputSize,hiddenSize,outputSize):
        super(RNN,self).__init__()
        self.hiddenSize=hiddenSize
        self.inputSize=inputSize
        self.layerN=2
        self.embedding = nn.Embedding(inputSize, hiddenSize)
        self.gru=nn.GRU(hiddenSize,hiddenSize,num_layers=2)
        self.o2o=nn.Linear(hiddenSize,outputSize)
        self.dropout=nn.Dropout(0.1)
        self.softmax=nn.LogSoftmax(dim=1)
    def forward(self,input,hidden):
        inputEmbedding = self.embedding(input.view(-1,1)).view(1, 1, -1)
        inputEmbedding = F.relu(inputEmbedding)
        output,hidden=self.gru(inputEmbedding,hidden)
        output=self.o2o(output)
        output=output[0]
        output=self.dropout(output)
        output=self.softmax(output)
        return output,hidden
    def initHidden(self):
        return torch.zeros(self.layerN,1,self.hiddenSize,device=device)

# nameGenerator

In [ ]:
class nameGenerator():
    def dataInit(self,fileName):
        nameFile = open(fileName,'r')
        self.c2n={}
        self.n2c={}
        self.c2n["EOS"]=0
        self.n2c[0]="EOS"
        self.c2n["SOS"]=1
        self.n2c[1]="SOS"
        self.nowIdx=1
        self.totChar=2
        self.names=[] #For example names[0]==['b',c,d]
        self.tNames=[] #Corresponding example: tNames[0]=[0,3,9,2,1] 
        
        for line in nameFile:
            name=[]
            tName=[]
            tName.append(self.c2n["SOS"])
            for c in line:
                if (self.c2n.get(c)==None):
                    self.nowIdx+=1
                    self.totChar+=1
                    self.c2n[c]=self.nowIdx
                    self.n2c[self.nowIdx]=c
                if (c!='\n'):
                    name.append(c)
                    tName.append(self.c2n[c])
            tName.append(self.c2n["EOS"])
            self.names.append(name)
            self.tNames.append(tName)
            
    def modelInit(self):
        self.nn=RNN(self.totChar,self.hiddenSize,self.totChar)
        if (device!=torch.device("cpu")):
            self.nn.cuda()
            
    def trainInit(self):
        self.optimizer = optim.Adagrad(self.nn.parameters())
        self.criterion=nn.NLLLoss()
        
        self.print_avgLoss=[]
        
    def parameterInit(self):
        self.iterN=13538*20
        self.hiddenSize=128
        self.maxLen=12
        self.sampleNumber=15
        
        self.printEveryIter=200
        self.startTime=time.time()
        
    def __init__(self,fileName):
        self.parameterInit()
        self.dataInit(fileName)
        self.modelInit()
        self.trainInit()
        
    def print(self):
        plt.figure()
        plt.plot(self.print_avgLoss)
        
    def trainOneIter(self,inputT,targetT):
        targetT.unsqueeze_(-1)

        loss=0
        hidden=self.nn.initHidden()
        for i in range(inputT.size(0)):
            output,hidden=self.nn(inputT[i],hidden)
            loss+=self.criterion(output,targetT[i])

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return output,loss.item()/inputT.size(0)
    
    def train(self):
        totLoss=0
        for iter in range(1,self.iterN+1):
            output,loss=self.trainOneIter(*randomTrainingData(self.tNames))
            totLoss+=loss
            if (iter%self.printEveryIter==0):
                avgLoss= (totLoss/self.printEveryIter)
                print("Iteration:{}, Training Time:{}, recentAvgLoss:{}".
                      format(iter,time.time()-self.startTime,avgLoss))
                totLoss=0
                self.print_avgLoss.append(avgLoss)
                
    def sample(self,prefix=''):
        with torch.no_grad():
            tPrefix=[self.c2n['SOS']]
            for c in prefix:
                tPrefix.append(self.c2n[c])
            inputT = toTensor(tPrefix)

            hidden = self.nn.initHidden()
            outputName=prefix
            for i in range(inputT.size(0)-1):
                output,hidden=self.nn(inputT[i],hidden)
            for i in range(self.maxLen):
                output,hidden=self.nn(inputT[-1],hidden)
                topv,topi=output.topk(1)
                topi=topi[0][0].item()
                if (topi==self.c2n["EOS"]):
                    break
                else:
                    outputName+=self.n2c[topi]
                inputT=toTensor([topi])
        return outputName
    def samples(self,prefix):
        self.sampleNumber=100
        resLst = [self.sample(prefix) for i in range(self.sampleNumber)]
        return list(set(resLst))

In [ ]:
originalFileName = "atcoderUserName.txt"
reverseFileName = reverseNameGenerate(originalFileName)

nameGenerator1 = nameGenerator(originalFileName)
nameGenerator1.train()
nameGenerator1.print()
nameGenerator2 = nameGenerator(reverseFileName)
nameGenerator2.train()
nameGenerator2.print()


In [ ]:
testString=''
print (nameGenerator1.samples(testString))
print (stringLstReverse(nameGenerator2.samples(testString[::-1])))

In [ ]:
print (nameGenerator1.c2n)